In [ ]:
import os
import qiskit
import torch
from qiskit_aer import AerSimulator
from qiskit import transpile, QuantumCircuit as QiskitQuantumCircuit
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torchxrayvision as xrv
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision

In [ ]:
QC_outputs = ['000', '001', '010', '011', '100', '101', '110', '111']

In [ ]:
class MammoDataset(Dataset):
    def __init__(self, root_dir):
        self.image_paths = []
        self.labels = []
        self.transform = torchvision.transforms.Compose([
            xrv.datasets.XRayCenterCrop(),
            xrv.datasets.XRayResizer(224)
        ])
        for label in ['0', '1']:
            folder_path = os.path.join(root_dir, label)
            for img_name in os.listdir(folder_path):
                self.image_paths.append(os.path.join(folder_path, img_name))
                self.labels.append(int(label))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB') 

        
        image = np.array(image)

        # Verifica a forma da imagem
        if len(image.shape) == 2:
            image = np.expand_dims(image, axis=-1)

        if self.transform:
            image = self.transform(image)

        # Converte a imagem para um tensor do PyTorch
        image = torch.tensor(image, dtype=torch.float32)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

In [ ]:
class DenseNetNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super(DenseNetNN, self).__init__()
        self.model = xrv.models.DenseNet(weights="all").features
        #self.embedding_layer = nn.Linear(1024, 128)

    def forward(self, x):
        x1 = self.model(x)

        return x1

In [ ]:
# Defines the quantum circuit
class QuantumCircuit:
    def __init__(self, n_qubits, backend, shots):
        # Define how many qubits we want
        self._circuit = QiskitQuantumCircuit(n_qubits)

        # List of qubits
        all_qubits = [i for i in range(n_qubits)]

        # Define 7 placeholder variables
        self.theta_0 = qiskit.circuit.Parameter('theta0')
        self.theta_1 = qiskit.circuit.Parameter('theta1')
        self.theta_2 = qiskit.circuit.Parameter('theta2')
        self.theta_3 = qiskit.circuit.Parameter('theta3')
        self.theta_4 = qiskit.circuit.Parameter('theta4')
        self.theta_5 = qiskit.circuit.Parameter('theta5')
        self.theta_6 = qiskit.circuit.Parameter('theta6')

        # Build the circuit
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta_0, all_qubits)
        self._circuit.cz(0, 1)
        self._circuit.cz(1, 2)
        self._circuit.ry(self.theta_1, 0)
        self._circuit.ry(self.theta_2, 1)
        self._circuit.ry(self.theta_3, 2)
        self._circuit.cz(0, 1)
        self._circuit.cz(1, 2)
        self._circuit.ry(self.theta_4, 0)
        self._circuit.ry(self.theta_5, 1)
        self._circuit.ry(self.theta_6, 2)
        self._circuit.measure_all()

        # Save these variables for later use
        self.backend = backend
        self.shots = shots

    def run(self, thetas):
        # Bind parameters to the circuit
        param_values = {
            self.theta_0: thetas[0],
            self.theta_1: thetas[1],
            self.theta_2: thetas[2],
            self.theta_3: thetas[3],
            self.theta_4: thetas[4],
            self.theta_5: thetas[5],
            self.theta_6: thetas[6]
        }

        # Create a bound circuit
        bound_circuit = self._circuit.assign_parameters(param_values)

        # Transpile the circuit for the specified backend
        transpiled_circuit = transpile(bound_circuit, self.backend)

        # Run the circuit
        job = self.backend.run(transpiled_circuit, shots=self.shots)
        result = job.result()

        # Get the counts and compute the expected values
        counts = result.get_counts(transpiled_circuit)

        expects = np.zeros(8)
        for k in range(8):
            key = QC_outputs[k]
            perc = counts.get(key, 0) / self.shots
            expects[k] = perc
        return expects

In [ ]:
#This class defines what our hybrid layer does. It allows it to go forward, and also backprop
class Hybrid(Function):

    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        parameter = []

        #A very scrapy but functional way of loading the parameters in this format
        for x in input:
            for y in x:
                parameter.append(float(y))
        # print(parameter)

        #Aka we run the input into our circuit
        expectation_z = ctx.quantum_circuit.run(parameter)  #Might need to change this when have multi-layer
        #Shoves to pytorch tesnor
        result = torch.tensor([expectation_z])
        #Save the input and result for backpropagation
        ctx.save_for_backward(input, result)
        #return output
        return result

    @staticmethod
    def backward(ctx, grad_output):
        #Regrabing the input and the output
        input, expectation_z = ctx.saved_tensors
        # input_list = np.array(input.tolist())
        # print(input)

        input_list = list(input)[0]
        # print(input_list)

        input_list = input_list.cpu()

        #A list of all the gradients
        gradients = torch.Tensor()

        #We're going to go through the inputs and then calculate the gradient for each one
        for i in range(7):
            #In order to get the gradients for each one, we shift one of the parameters a tiny bit and find the difference in the outputs... Since we can't do backprop on it, we use something called shift parameter
            shift_right = np.array(input_list.detach().clone())
            shift_left = np.array(input_list.detach().clone())
            shift_right[i] = shift_right[i] + ctx.shift
            shift_left[i] = shift_left[i] - ctx.shift

            #So we take the shifted ones and just compute it
            expectation_right = ctx.quantum_circuit.run(shift_right)
            expectation_left = ctx.quantum_circuit.run(shift_left)
            #Gradient = appox the difference (division by 2*shift isn't necessary since it'll just be a scaled version (which can be counteracted by lr))
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            #Append the gradient to the meta list
            gradients = torch.cat((gradients, gradient.float()))

        #Format everything
        result = torch.Tensor(gradients)
        result = result.float()
        result = result.T

        #Just a random varaible that made my life easier when debugging
        Fixer = grad_output.float()
        Fixer = Fixer.T

        #Find the gradients at last!!!
        h = result * Fixer
        h = h.cuda()
        #then return it for backprop
        return h, None, None

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.classical = DenseNetNN()
        self.middle = nn.Sequential(
            nn.Linear(1024 * 7 * 7, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 7)
        )
        sim = AerSimulator()
        self.quantum = Hybrid(sim, 1000, np.pi / 2)
        self.end = nn.Sequential(
            nn.Linear(8, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        # Passa as entradas pelo modelo
        x = self.classical(x)
        x = nn.Flatten(start_dim=1)(x)
        x = self.middle(x)
        theta = x
        x = self.quantum(x)
        x = x.type(dtype=torch.cuda.FloatTensor)
        x = self.end(x.float())
        x = nn.Flatten(start_dim=1)(x)
        return x, theta

In [ ]:
train_dir = 'C:\\Users\\Win10\\PycharmProjects\\QuantumResearch\\INbreast\\data\\png\\train'
test_dir = 'C:\\Users\\Win10\\PycharmProjects\\QuantumResearch\\INBreast\\data\\png\\test'

# Instancia os datasets
train_dataset = MammoDataset(root_dir=train_dir)
test_dataset = MammoDataset(root_dir=test_dir)

# Cria os DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = Net().cuda()
loss_func = nn.BCEWithLogitsLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 1
loss_list = []
thetas = []

In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = []

    # Adiciona barra de progresso para o loop dos batches
    for batch_idx, (image, label) in enumerate(train_loader):
        label = label.unsqueeze(1)

        # Converta os tensores para o tipo correto
        image = image.cuda()
        label = label.cuda().type(torch.cuda.FloatTensor)

        optimizer.zero_grad()  # Zere os gradientes

        # Passe as imagens pelo modelo
        prediction, theta = model(image)

        # Calcule a perda
        loss = loss_func(prediction, label)

        # Propagação para trás e atualização dos pesos
        loss.backward()
        optimizer.step()

        # Armazene as informações
        thetas.append(theta)
        total_loss.append(loss.item())
        print(f"  Batch {batch_idx + 1} concluído com loss: {loss.item():.4f}")
        
    avg_loss = np.mean(total_loss)
    loss_list.append(avg_loss)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")

    model_save_path = f"model_epoch_{epoch}.pth"
    torch.save(model.state_dict(), model_save_path)


In [ ]:
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for image, label in test_loader:
        image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.LongTensor)
        try:
            prediction, _ = model(image)
            predicted_class = prediction.argmax(dim=1)
            all_labels.extend(label.cpu().numpy())
            all_predictions.extend(predicted_class.cpu().numpy())
        except:
            continue

# Calcular acurácia
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Acurácia: {accuracy:.4f}")

In [ ]:
# Gerar matriz de confusão
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Plotar matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'],
            yticklabels=['Negativo', 'Positivo'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()